In [1]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline
from tqdm import tqdm

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
raw_df = pd.read_csv('cat_de_3year.csv', encoding = "utf-8")#, nrows = 100000)
#raw_df = pd.read_csv('cat_de_15-18.csv', encoding = "utf-8")
#raw_df.head(20)

In [4]:
print('min date is {}'.format(min(raw_df['TDT_T_KEY_ORDER_DATE'])))
print('max date is {}'.format(max(raw_df['TDT_T_KEY_ORDER_DATE'])))

min date is 01-APR-15
max date is 31-OCT-17


# Let us create a data frame that contains only required data for future basic recommender system
### As of now we are not taking into account session information
> Logic for defining the strength of interaction of the user with the item would be
1. Bought an item $(0,2]$ times - weight will be "1.0"
2. Bought an item $(2,5]$ times - weight will be "2.0"
3. Bought an item $(5, +\infty)$ times - weight will be "3.0"
> I can think of a functional weight for the counts of times the item was boubht. TODO

### Required columns are
* CUS_N_KEY_CUSTOMER == personId
* PPR_N_PHY_PRODUCT_ID = contentId
* eventStrength = depends on the overall history of the customer-item interactions. _no basket or date informaiton is used_


In [19]:
## to start let us take a sample of our data
sample_df = raw_df.sample(3000000)
sample_df.head()

,SCM_N_ORDER_POS_NR,SCM_N_ORDER_NR,SCM_N_ORDER_NR_NC,TDT_T_KEY_ORDER_DATE,ART_N_KEY_ARTICLE,PPR_N_KEY_PHYS_PRODUCT,CUS_N_KEY_CUSTOMER,SCM_D_QUANTITY,PPR_N_PHY_ARTICLE_ID,PPR_N_PHY_PRODUCT_ID
2854011,211244425,50305642,0,01-MAR-15,1438193,70596985,10803110,1.0000,63206,9828
32731418,380500503,89162389,89162389,18-JUN-17,2190020,123775564,17096962,0.0364,73148,49878
9149454,236614527,56187909,0,03-AUG-15,1398017,69976150,9020143,6.0000,62786,45441
5784658,233310054,55415028,0,14-JUL-15,1634764,14908,5272420,0.5004,20346,12753
5423421,238894360,56717834,0,17-AUG-15,1305942,62777224,6342878,1.0000,61231,4904


In [20]:
# remove non-used columns
sample_df.drop(['SCM_N_ORDER_POS_NR','SCM_N_ORDER_NR_NC', 'SCM_N_ORDER_NR_NC', 'PPR_N_KEY_PHYS_PRODUCT', 'SCM_N_ORDER_NR', 'ART_N_KEY_ARTICLE','PPR_N_PHY_ARTICLE_ID'], axis=1, inplace=True)
sample_df.head()

,TDT_T_KEY_ORDER_DATE,CUS_N_KEY_CUSTOMER,SCM_D_QUANTITY,PPR_N_PHY_PRODUCT_ID
2854011,01-MAR-15,10803110,1.0000,9828
32731418,18-JUN-17,17096962,0.0364,49878
9149454,03-AUG-15,9020143,6.0000,45441
5784658,14-JUL-15,5272420,0.5004,12753
5423421,17-AUG-15,6342878,1.0000,4904


In [21]:
# this is not needed since we are simply counting prod id
# sample_more_1_interaction = sample_df.groupby(['CUS_N_KEY_CUSTOMER','PPR_N_PHY_PRODUCT_ID']).filter(lambda x: x['SCM_D_QUANTITY'].unique().size > 1 )

In [22]:
#sample_more_1_interaction.head()

In [29]:
tmp = sample_df.groupby(['CUS_N_KEY_CUSTOMER', 'PPR_N_PHY_PRODUCT_ID']).count()
tmp.reset_index(level=tmp.index.names, inplace=True)
interactions_df = tmp.rename(index=str, columns = {'CUS_N_KEY_CUSTOMER': 'personId', 
                                                   'PPR_N_PHY_PRODUCT_ID': 'contentId', 
                                                   'TDT_T_KEY_ORDER_DATE': 'date', 
                                                   'SCM_D_QUANTITY': 'eventStrength'}).drop('date', axis=1)
#interactions_df = interactions_df[interactions_df['eventStrength']>2]
interactions_df.head()

,personId,contentId,eventStrength
0,22,9831,1
1,22,15896,1
2,22,28038,1
3,22,44561,1
4,22,44956,1


In [30]:
# Visualize pairplot of df
# sns.pairplot(interactions_df, hue='eventStrength');

Recommender systems have a problem known as user cold-start, in which is hard do provide personalized recommendations for users with none or a very few number of consumed items, due to the lack of information to model their preferences.
For this reason, we are keeping in the dataset only users with at leas 5 interactions.

In [38]:
users_interactions_count_df = interactions_df.groupby(['personId', 'contentId']).size().groupby('personId').size()
print('# users: %d' % len(users_interactions_count_df))
users_with_enough_interactions_df = users_interactions_count_df[users_interactions_count_df >= 20].reset_index()[['personId']]
print('# users with at least 5 interactions: %d' % len(users_with_enough_interactions_df))

# users: 604378
# users with at least 5 interactions: 10073


In [39]:
print('# of interactions: %d' % len(interactions_df))
interactions_from_selected_users_df = interactions_df.merge(users_with_enough_interactions_df, 
               how = 'right',
               left_on = 'personId',
               right_on = 'personId')
print('# of interactions from users with at least 5 interactions: %d' % len(interactions_from_selected_users_df))

# of interactions: 2288251
# of interactions from users with at least 5 interactions: 277523


In [40]:
####
# This must be changed
###
interactions_full_df = interactions_from_selected_users_df

## Evaluation
Evaluation is important for machine learning projects, because it allows to compare objectivelly different algorithms and hyperparameter choices for models.

One key aspect of evaluation is to ensure that the trained model generalizes for data it was not trained on, using *Cross-validation techniques*. We are using here a simple cross-validation approach named *holdout*, in which a random data sample (20% in this case) are kept aside in the training process, and exclusively used for evaluation. All evaluation metrics reported here are computed using the *test set*.

_Ps. A more robust evaluation approach could be to split train and test sets by a reference date, where the train set is composed by all interactions before that date, and the test set are interactions after that date. For the sake of simplicity, we chose the first random approach for this notebook, but you may want to try the second approach to better simulate how the recsys would perform in production predicting "future" users interactions._

In [41]:
interactions_train_df, interactions_test_df = train_test_split(interactions_full_df,
                                   stratify=interactions_full_df['personId'], 
                                   test_size=0.20,
                                   random_state=42)

print('# interactions on Train set: %d' % len(interactions_train_df))
print('# interactions on Test set: %d' % len(interactions_test_df))

# interactions on Train set: 222018
# interactions on Test set: 55505


In Recommender Systems, there are a set metrics commonly used for evaluation. We chose to work with *Top-N* accuracy metrics, which evaluates the accuracy of the top recommendations provided to a user, comparing to the items the user has actually interacted in test set.

This evaluation method works as follows:

* For each user
    * For each item the user has interacted in test set
        * Sample 100 other items the user has never interacted. Ps. Here we naively assume those non interacted items are not relevant to the user, which might not be true, as the user may simply not be aware of those not interacted items. But let's keep this assumption.
        * Ask the recommender model to produce a ranked list of recommended items, from a set composed one interacted item and the 100 non-interacted ("non-relevant!) items
        * Compute the Top-N accuracy metrics for this user and interacted item from the recommendations ranked list
* Aggregate the global Top-N accuracy metrics

The Top-N accuracy metric choosen was *Recall@N* which evaluates whether the interacted item is among the top N items (hit) in the ranked list of 101 recommendations for a user.
Ps. Other popular ranking metrics are *NDCG@N* and *MAP@N*, whose score calculation takes into account the position of the relevant item in the ranked list (max. value if relevant item is in the first position). You can find a reference to implement this metrics in this post.

In [42]:
#Indexing by personId to speed up the searches during evaluation
interactions_full_indexed_df = interactions_full_df.set_index('personId')
interactions_train_indexed_df = interactions_train_df.set_index('personId')
interactions_test_indexed_df = interactions_test_df.set_index('personId')

In [43]:
def get_items_interacted(person_id, interactions_df):
    # Get the user's data and merge in the movie information.
    interacted_items = interactions_df.loc[person_id]['contentId']
    return set(interacted_items if type(interacted_items) == pd.Series else [interacted_items])

In [99]:
#Top-N accuracy metrics consts
EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS = 100

class ModelEvaluator:


    def get_not_interacted_items_sample(self, person_id, sample_size, seed=42):
        interacted_items = get_items_interacted(person_id, interactions_full_indexed_df)
        all_items = set(articles_df['contentId'])
        non_interacted_items = all_items - interacted_items

        random.seed(seed)
        non_interacted_items_sample = random.sample(non_interacted_items, sample_size)
        return set(non_interacted_items_sample)

    def _verify_hit_top_n(self, item_id, recommended_items, topn):        
            try:
                index = next(i for i, c in enumerate(recommended_items) if c == item_id)
            except:
                index = -1
            hit = int(index in range(0, topn))
            return hit, index

    def evaluate_model_for_user(self, model, person_id):
        #Getting the items in test set
        interacted_values_testset = interactions_test_indexed_df.loc[person_id]
        if type(interacted_values_testset['contentId']) == pd.Series:
            person_interacted_items_testset = set(interacted_values_testset['contentId'])
        else:
            person_interacted_items_testset = set([int(interacted_values_testset['contentId'])])  
        interacted_items_count_testset = len(person_interacted_items_testset) 

        #Getting a ranked recommendation list from a model for a given user
        person_recs_df = model.recommend_items(person_id, 
                                               items_to_ignore=get_items_interacted(person_id, 
                                                                                    interactions_train_indexed_df), 
                                               topn=10000000000)

        hits_at_5_count = 0
        hits_at_10_count = 0
        #For each item the user has interacted in test set
        for item_id in person_interacted_items_testset:
            #Getting a random sample (100) items the user has not interacted 
            #(to represent items that are assumed to be no relevant to the user)
            non_interacted_items_sample = self.get_not_interacted_items_sample(person_id, 
                                                                          sample_size=EVAL_RANDOM_SAMPLE_NON_INTERACTED_ITEMS, 
                                                                          seed=item_id%(2**32))

            #Combining the current interacted item with the 100 random items
            items_to_filter_recs = non_interacted_items_sample.union(set([item_id]))

            #Filtering only recommendations that are either the interacted item or from a random sample of 100 non-interacted items
            valid_recs_df = person_recs_df[person_recs_df['contentId'].isin(items_to_filter_recs)]                    
            valid_recs = valid_recs_df['contentId'].values
            #Verifying if the current interacted item is among the Top-N recommended items
            hit_at_5, index_at_5 = self._verify_hit_top_n(item_id, valid_recs, 5)
            hits_at_5_count += hit_at_5
            hit_at_10, index_at_10 = self._verify_hit_top_n(item_id, valid_recs, 10)
            hits_at_10_count += hit_at_10

        #Recall is the rate of the interacted items that are ranked among the Top-N recommended items, 
        #when mixed with a set of non-relevant items
        recall_at_5 = hits_at_5_count / float(interacted_items_count_testset)
        recall_at_10 = hits_at_10_count / float(interacted_items_count_testset)

        person_metrics = {'hits@5_count':hits_at_5_count, 
                          'hits@10_count':hits_at_10_count, 
                          'interacted_count': interacted_items_count_testset,
                          'recall@5': recall_at_5,
                          'recall@10': recall_at_10}
        return person_metrics

    def evaluate_model(self, model):
        #print('Running evaluation for users')
        people_metrics = []
        i=0
        for idx, person_id in enumerate(list(interactions_test_indexed_df.index.unique().values)):
            i += 1
            if i == 1000:
                break
            #if idx % 100 == 0 and idx > 0:
            #    print('%d users processed' % idx)
            person_metrics = self.evaluate_model_for_user(model, person_id)  
            person_metrics['_person_id'] = person_id
            people_metrics.append(person_metrics)
        print('%d users processed' % idx)

        detailed_results_df = pd.DataFrame(people_metrics) \
                            .sort_values('interacted_count', ascending=False)
        
        global_recall_at_5 = detailed_results_df['hits@5_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        global_recall_at_10 = detailed_results_df['hits@10_count'].sum() / float(detailed_results_df['interacted_count'].sum())
        
        global_metrics = {'modelName': model.get_model_name(),
                          'recall@5': global_recall_at_5,
                          'recall@10': global_recall_at_10}    
        return global_metrics, detailed_results_df
    
model_evaluator = ModelEvaluator()  

## Popularity model

A common (and usually hard-to-beat) baseline approach is the Popularity model. This model is not actually personalized - it simply recommends to a user the most popular items that the user has not previously consumed. As the popularity accounts for the "wisdom of the crowds", it usually provides good recommendations, generally interesting for most people.
Ps. The main objective of a recommender system is to leverage the long-tail items to the users with very specific interests, which goes far beyond this simple technique.

In [100]:
#Computes the most popular items
item_popularity_df = interactions_full_df.groupby('contentId')['eventStrength'].sum().sort_values(ascending=False).reset_index()
item_popularity_df.head(10)

,contentId,eventStrength
0,38333,5698
1,42948,5697
2,11809,5433
3,10178,5070
4,3187,4067
5,23513,4061
6,7785,3709
7,39751,3683
8,17930,3613
9,46365,3530


In [101]:
class PopularityRecommender:
    
    MODEL_NAME = 'Popularity'
    
    def __init__(self, popularity_df, items_df=None):
        self.popularity_df = popularity_df
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # Recommend the more popular items that the user hasn't seen yet.
        recommendations_df = self.popularity_df[~self.popularity_df['contentId'].isin(items_to_ignore)] \
                               .sort_values('eventStrength', ascending = False) \
                               .head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'contentId', 
                                                          right_on = 'contentId')[['eventStrength', 'contentId', 'title', 'url', 'lang']]


        return recommendations_df
    
# sample_df
# popularity_model = PopularityRecommender(item_popularity_df, articles_df)
articles_df = sample_df.rename(index=str, columns = {'CUS_N_KEY_CUSTOMER': 'personId', 
                                                   'PPR_N_PHY_PRODUCT_ID': 'contentId'})
popularity_model = PopularityRecommender(item_popularity_df, articles_df)

Here we perform the evaluation of the Popularity model, according to the method described above.
It achieved the Recall@5 of ????, which means that about ??% of interacted items in test set were ranked by Popularity model among the top-5 items (from lists with 100 random items). And Recall@10 was even higher (???%), as expected.
It might be surprising to you that usually Popularity models could perform so well!

In [102]:
print('Evaluating Popularity recommendation model...')
pop_global_metrics, pop_detailed_results_df = model_evaluator.evaluate_model(popularity_model)
print('\nGlobal metrics:\n%s' % pop_global_metrics)
pop_detailed_results_df.head(10)

Evaluating Popularity recommendation model...
999 users processed

Global metrics:
{'modelName': 'Popularity', 'recall@5': 0.3800224107571634, 'recall@10': 0.5428205538658556}


,_person_id,hits@10_count,hits@5_count,interacted_count,recall@10,recall@5
608,8711842,14,9,30,0.466667,0.300000
883,8882040,15,10,23,0.652174,0.434783
781,18461914,8,5,22,0.363636,0.227273
265,9705814,10,8,21,0.476190,0.380952
305,6391658,10,3,21,0.476190,0.142857
838,9705943,10,6,20,0.500000,0.300000
477,871102,10,9,20,0.500000,0.450000
20,2354109,10,8,20,0.500000,0.400000
559,14736454,6,5,19,0.315789,0.263158
358,1037716,8,5,18,0.444444,0.277778


#### Content-Based Filtering model is NOT applicable to the data we are working on with right now
Content-based filtering approaches leverage description or attributes from items the user has interacted to recommend similar items. It depends only on the user previous choices, making this method robust to avoid the cold-start problem. For textual items, like articles, news and books, it is simple to use the raw text to build item profiles and user profiles.
Here we are using a very popular technique in information retrieval (search engines) named TF-IDF. This technique converts unstructured text into a vector structure, where each word is represented by a position in the vector, and the value measures how relevant a given word is for an article. As all items will be represented in the same Vector Space Model, it is to compute similarity between articles.

See this presentation (from slide 30) for more information on TF-IDF and Cosine similarity.

## Collaborative Filtering model

Collaborative Filtering (CF) has two main implementation strategies:

* Memory-based: This approach uses the memory of previous users interactions to compute users similarities based on items they've interacted (user-based approach) or compute items similarities based on the users that have interacted with them (item-based approach).
A typical example of this approach is User Neighbourhood-based CF, in which the top-N similar users (usually computed using Pearson correlation) for a user are selected and used to recommend items those similar users liked, but the current user have not interacted yet. This approach is very simple to implement, but usually do not scale well for many users. A nice Python implementation of this approach in available in Crab.
* Model-based: This approach, models are developed using different machine learning algorithms to recommend items to users. There are many model-based CF algorithms, like neural networks, bayesian networks, clustering models, and latent factor models such as Singular Value Decomposition (SVD) and, probabilistic latent semantic analysis.

### Matrix Factorization

Latent factor models compress user-item matrix into a low-dimensional representation in terms of latent factors. One advantage of using this approach is that instead of having a high dimensional matrix containing abundant number of missing values we will be dealing with a much smaller matrix in lower-dimensional space.
A reduced presentation could be utilized for either user-based or item-based neighborhood algorithms that are presented in the previous section. There are several advantages with this paradigm. It handles the sparsity of the original matrix better than memory based ones. Also comparing similarity on the resulting matrix is much more scalable especially in dealing with large sparse datasets.

Here we a use popular latent factor model named Singular Value Decomposition (SVD). There are other matrix factorization frameworks more specific to CF you might try, like surprise, mrec or python-recsys. We chose a SciPy implemenation of SVD because it is available on Kaggle kernels.
Ps. See an example of SVD on a movies dataset in this blog post.

An important decision is the number of factors to factor the user-item matrix. The higher the number of factors, the more precise is the factorization in the original matrix reconstructions. Therefore, if the model is allowed to memorize too much details of the original matrix, it may not generalize well for data it was not trained on. Reducing the number of factors increases the model generalization.

In [103]:
#Creating a sparse pivot table with users in rows and items in columns
users_items_pivot_matrix_df = interactions_train_df.pivot(index='personId', 
                                                          columns='contentId', 
                                                          values='eventStrength').fillna(0)

users_items_pivot_matrix_df.head(10)

contentId,1078,1080,1092,1131,1164,1168,1234,1404,1408,1452,...,55783,55784,55891,56168,56169,56173,56978,56979,57073,57319
personId,,,,,,,,,,,,,,,,,,,,,
84,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
205,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
413,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
610,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1525,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1682,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3133,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3605,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [104]:
users_items_pivot_matrix = users_items_pivot_matrix_df.as_matrix()
users_items_pivot_matrix[:10]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [105]:
users_ids = list(users_items_pivot_matrix_df.index)
users_ids[:10]

[84, 205, 413, 610, 1375, 1525, 1682, 3133, 3605, 5755]

In [106]:
#The number of factors to factor the user-item matrix.
NUMBER_OF_FACTORS_MF = 15
#Performs matrix factorization of the original user item matrix
U, sigma, Vt = svds(users_items_pivot_matrix, k = NUMBER_OF_FACTORS_MF)

In [107]:
U.shape

(10073, 15)

In [108]:
(15, 2926)

(15, 2926)

In [109]:
sigma = np.diag(sigma)
sigma.shape

(15, 15)

After the factorization, we try to to reconstruct the original matrix by multiplying its factors. The resulting matrix is not sparse any more. It was generated predictions for items the user have not yet interaction, which we will exploit for recommendations.

In [110]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) 
all_user_predicted_ratings

array([[ 4.10708040e-03,  2.98897809e-03,  2.61832280e-03, ...,
         5.86694056e-03,  2.68183766e-03, -1.72378413e-05],
       [ 6.82102344e-04,  6.34645457e-03,  3.18245572e-04, ...,
         2.75570449e-03,  2.65867103e-03,  2.94945601e-05],
       [ 2.51361944e-03,  7.64704373e-03,  3.82205574e-03, ...,
         7.40816982e-03,  9.50169822e-03, -2.59232448e-05],
       ...,
       [ 1.18021771e-03,  3.26362118e-03,  4.88287336e-04, ...,
         1.77553000e-03,  2.05749490e-03,  3.23059470e-05],
       [ 1.85654551e-03,  5.89303241e-03,  6.95906733e-04, ...,
         2.46159925e-03,  2.99086842e-03,  3.49726209e-05],
       [ 2.52736705e-03, -6.58988996e-03, -1.08750322e-03, ...,
        -4.25709551e-04,  2.24759654e-04, -1.83577840e-05]])

In [111]:
#Converting the reconstructed matrix back to a Pandas dataframe
cf_preds_df = pd.DataFrame(all_user_predicted_ratings, columns = users_items_pivot_matrix_df.columns, index=users_ids).transpose()
cf_preds_df.head(10)

,84,205,413,610,1375,1525,1682,3133,3605,5755,...,17155814,17269843,17326637,17672952,18461809,18461914,18533637,18802223,19570394,20055617
contentId,,,,,,,,,,,,,,,,,,,,,
1078,0.004107,0.000682,0.002514,-0.000270,0.001344,0.000941,0.002124,0.000003,0.001134,0.005660,...,0.002900,0.000936,0.003947,0.000770,-0.008155,0.001643,0.002511,0.001180,0.001857,0.002527
1080,0.002989,0.006346,0.007647,-0.000237,0.001820,0.001667,0.008600,0.000251,0.001636,0.021186,...,0.003530,0.000882,0.010748,0.002065,0.248425,0.218414,-0.002826,0.003264,0.005893,-0.006590
1092,0.002618,0.000318,0.003822,-0.000891,0.000319,0.001391,0.002050,0.000557,0.000539,0.001345,...,0.000547,0.000470,0.001767,0.000207,0.009591,0.007738,-0.000398,0.000488,0.000696,-0.001088
1131,0.001200,0.000241,0.001048,-0.000298,0.000546,0.000081,0.000445,0.001080,0.000393,0.000331,...,0.000310,0.000710,0.001002,0.000567,0.004375,0.007908,0.001036,0.000351,0.000589,0.001627
1164,0.001233,0.000534,0.000665,0.000019,0.000931,0.001620,0.002820,0.000691,0.000777,0.000964,...,0.001682,0.000721,0.004522,0.000731,0.019745,0.017944,-0.000861,0.001758,0.003584,-0.001203
1168,0.000121,0.000119,0.000234,-0.000089,0.000006,-0.000011,0.000125,-0.000002,0.000056,0.000064,...,-0.000004,0.000037,0.000061,0.000273,0.000974,0.001212,0.000159,0.000079,0.000066,0.000074
1234,0.000029,-0.000184,0.000168,-0.000097,0.000147,0.000147,0.000024,0.000220,0.000193,-0.000950,...,0.000481,0.000112,-0.000006,0.000178,-0.000571,0.000508,0.000618,0.000075,0.000177,0.000560
1404,0.000234,0.000413,0.001492,-0.000161,0.000467,0.000281,0.000795,0.000657,0.000106,-0.000082,...,0.000337,0.000346,0.000582,0.000024,-0.002326,-0.002525,-0.000212,0.000240,0.000344,-0.000205
1408,0.000915,0.000910,0.001893,-0.000384,0.001336,0.001572,0.002850,0.000434,0.000625,0.002919,...,0.002677,0.000486,0.003589,0.000163,0.013812,0.010383,-0.001113,0.001475,0.002849,-0.001958


In [112]:
len(cf_preds_df.columns)

10073

In [113]:
class CFRecommender:
    
    MODEL_NAME = 'Collaborative Filtering'
    
    def __init__(self, cf_predictions_df, items_df=None):
        self.cf_predictions_df = cf_predictions_df
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # Get and sort the user's predictions
        sorted_user_predictions = self.cf_predictions_df[user_id].sort_values(ascending=False) \
                                    .reset_index().rename(columns={user_id: 'recStrength'})

        # Recommend the highest predicted rating movies that the user hasn't seen yet.
        recommendations_df = sorted_user_predictions[~sorted_user_predictions['contentId'].isin(items_to_ignore)] \
                               .sort_values('recStrength', ascending = False) \
                               .head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'contentId', 
                                                          right_on = 'contentId')[['recStrength', 'contentId', 'title', 'url', 'lang']]


        return recommendations_df

articles_df = sample_df.rename(index=str, columns = {'CUS_N_KEY_CUSTOMER': 'personId', 
                                               'PPR_N_PHY_PRODUCT_ID': 'contentId'})
cf_recommender_model = CFRecommender(cf_preds_df, articles_df)

In [114]:
print('Evaluating Collaborative Filtering (SVD Matrix Factorization) model...')
cf_global_metrics, cf_detailed_results_df = model_evaluator.evaluate_model(cf_recommender_model)
print('\nGlobal metrics:\n%s' % cf_global_metrics)
cf_detailed_results_df.head(10)

Evaluating Collaborative Filtering (SVD Matrix Factorization) model...
999 users processed

Global metrics:
{'modelName': 'Collaborative Filtering', 'recall@5': 0.4041940131263006, 'recall@10': 0.5674723867456379}


,_person_id,hits@10_count,hits@5_count,interacted_count,recall@10,recall@5
608,8711842,25,20,30,0.833333,0.666667
883,8882040,17,12,23,0.739130,0.521739
781,18461914,18,16,22,0.818182,0.727273
265,9705814,19,15,21,0.904762,0.714286
305,6391658,12,9,21,0.571429,0.428571
838,9705943,14,14,20,0.700000,0.700000
477,871102,9,7,20,0.450000,0.350000
20,2354109,7,5,20,0.350000,0.250000
559,14736454,11,5,19,0.578947,0.263158
358,1037716,11,7,18,0.611111,0.388889
